In [1]:
%%HTML
<style type='text/css'>
    *{
        # background-color:#E3EDCD;
        # background-color:black;
        # color:white;
        
    }
    h1{
        color:#1976d2;
    }
    h2{
        color:#f57c00;
    }
    h3{
        color:#ba37ff;
    }
    h4{
        color:green;
    }
    table{
        border:1px solid black !important;
        border-collapse:collapse !important;
    }
    th{
        background-color:blueviolet !important;
        text-align:center;
        color:white;
    }
    th,td{
        border:0.1px solid black !important;
        transition:0.2s all liner;
        
    }
    td:hover{
        transform:scale(1.1);
        background-color:orange;
        color:blueviolet;
    }
    .raw{
        white-space:pre;
    }
    .important{
        color:red;
    }
</style>

## 房价预测_回归问题

前面两个例子都是分类问题，其目标是预测输入数据点所对应的单一离散的标签。另一种
常见的机器学习问题是回归问题，它预测一个连续值而不是离散的标签，例如，根据气象数据
预测明天的气温，或者根据软件说明书预测完成软件项目所需要的时间。

### 3.6.1　波士顿房价数据集


本节将要预测 20 世纪 70 年代中期波士顿郊区房屋价格的中位数，已知当时郊区的一些数
据点，比如犯罪率、当地房产税率等。本节用到的数据集与前面两个例子有一个有趣的区别。
它包含的数据点相对较少，只有 506 个，分为 404 个训练样本和 102 个测试样本。输入数据的
每个特征（比如犯罪率）都有不同的取值范围。例如，有些特性是比例，取值范围为 0\~1；有
的取值范围为 1\~12；还有的取值范围为 0\~100，等等。

#### 代码清单 3-24　加载波士顿房价数据

In [2]:
from keras.datasets import boston_housing

In [3]:
(train_data,train_targets),(test_data,test_targets)\
=boston_housing.load_data()

57026/57026 ━━━━━━━━━━━━━━━━━━━━ 0s 8us/step 


In [4]:
train_data.shape

(404, 13)

In [5]:
test_data.shape

(102, 13)

如你所见，我们有 404 个训练样本和 102 个测试样本，每个样本都有 13 个数值特征，比如
人均犯罪率、每个住宅的平均房间数、高速公路可达性等。  
目标是房屋价格的中位数，单位是千美元。


In [6]:
train_targets

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

房价大都在 10 000~50 000 美元。如果你觉得这很便宜，不要忘记当时是 20 世纪 70 年代中
期，而且这些价格没有根据通货膨胀进行调整。

### 3.6.2　准备数据

将取值范围差异很大的数据输入到神经网络中，这是有问题的。网络可能会自动适应这种
取值范围不同的数据，但学习肯定变得更加困难。对于这种数据，普遍采用的最佳实践是对每
个特征做标准化，即对于输入数据的每个特征（输入数据矩阵中的列），减去特征平均值，再除
以标准差，这样得到的特征平均值为 0，标准差为 1。用 Numpy 可以很容易实现标准化

#### 代码清单 3-25　数据标准化

In [10]:
mean = train_data.mean(axis=0) #首先计算每行数据的平均值
train_data -= mean #从训练数据的每一列中减去对应的平均值。
# 这一步操作后，每个特征的新平均值都会变为0。这种中心化处理有助于算法更快地收敛
std = train_data.std(axis=0)
# 计算标准化后的训练数据每一列的标准差，并将其存储在变量std中。
# 同样，axis=0表示计算每一列的标准差。
# 注意这里使用的是已经中心化处理后的训练数据来计算标准差。

train_data /= std
# 将训练数据的每一列除以对应的标准差。这样做是为了使每个特征的方差都变为1。经过这一步骤，训练数据就被标准化了，具有零均值和单位方差。

test_data -= mean
test_data /= std

注意，用于测试数据标准化的均值和标准差都是在训练数据上计算得到的。在工作流程中，
你不能使用在测试数据上计算得到的任何结果，即使是像数据标准化这么简单的事情也不行

### 3.6.3　构建网络

由于样本数量很少，我们将使用一个非常小的网络，其中包含两个隐藏层，每层有 64 个单
元。一般来说，训练数据越少，过拟合会越严重，而<span class='important'>较小的网络可以降低过拟合。</p>

#### 代码清单 3-26　模型定义

In [11]:
from keras import models,layers,Input

In [12]:
def build_model():
    model = models.Sequential()
    model.add(Input(shape=(train_data.shape[1],)))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(1))
    model.compile(
        optimizer = 'rmsprop',
        loss = 'mse',
        metrics=['mae']
    )
    return model

网络的最后一层只有一个单元，没有激活，是一个线性层。<span class='important'>这是标量回归（标量回归是预
测单一连续值的回归）的典型设置。添加激活函数将会限制输出范围。例如，如果向最后一层
添加 sigmoid 激活函数，网络只能学会预测 0~1 范围内的值。这里最后一层是纯线性的，所以
网络可以学会预测任意范围内的值。  </span>           
注意，编译网络用的是 mse 损失函数，即均方误差（MSE，mean squared error），预测值与
目标值之差的平方。这是回归问题常用的损失函数。            
在训练过程中还监控一个新指标：平均绝对误差（MAE，mean absolute error）。它是预测值
与目标值之差的绝对值。比如，如果这个问题的 MAE 等于 0.5，就表示你预测的房价与实际价
格平均相差 500 (即0.5$\times$1千美元\[这个数据集中的单位\])美元。         

In [13]:
model = build_model()
model.fit(train_data,train_targets,
         epochs = 80,batch_size = 16,verbose=0)


In [14]:
test_mse_score,test_mae_score = model.evaluate(test_data,test_targets)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 15.2580 - mae: 2.6668 


In [15]:
test_mae_score

2.8692893981933594

你预测的房价还是和实际价格相差约 2800 美元。

### 3.6.5　小结

下面是你应该从这个例子中学到的要点。       
 回归问题使用的损失函数与分类问题不同。回归常用的损失函数是均方误差（MSE）。    
 同样，回归问题使用的评估指标也与分类问题不同。显而易见，精度的概念不适用于回
归问题。常见的回归指标是平均绝对误差（MAE）。        
 如果输入数据的特征具有不同的取值范围，应该先进行预处理，对每个特征单独进行
缩放。        
 如果可用的数据很少，使用 K 折验证可以可靠地评估模型。      
 如果可用的训练数据很少，最好使用隐藏层较少（通常只有一到两个）的小型网络，以
避免严重的过拟合。      

本章小结
 现在你可以处理关于向量数据最常见的机器学习任务了：二分类问题、多分类问题和标
量回归问题。前面三节的“小结”总结了你从这些任务中学到的要点。     
 在将原始数据输入神经网络之前，通常需要对其进行预处理。       
 如果数据特征具有不同的取值范围，那么需要进行预处理，将每个特征单独缩放。       
 随着训练的进行，神经网络最终会过拟合，并在前所未见的数据上得到更差的结果。        
 如果训练数据不是很多，应该使用只有一两个隐藏层的小型网络，以避免严重的过拟合。      
 如果数据被分为多个类别，那么中间层过小可能会导致信息瓶颈。     
 回归问题使用的损失函数和评估指标都与分类问题不同。    
 如果要处理的数据很少，K 折验证有助于可靠地评估模型。        